# Models for the 2AFC auditory triangles task

## Alexandre Filipowicz & Adrian Radillo

### April 19th, 2019

The goal for this notebook is to create working models for the auditory 2AFC task. The goal of the project is to have people behave with different levels of mental model complexity. We will put together a more formal definition in another document, but briefly complexity is being characterized as the amount of past information being encoded to predict the future (cf., Bialek et al., 2001, Neural Computation).

Our main goal is to come up with task conditions that manipulate a person's model complexity while being exposed to very similar stimuli (i.e., use the same stimuli in more/less complex ways).

There are three conditions we would like to consider (at least, as a start):

- **Passive condition:** This is the condition that requires the simplest model as no information needs to be encoded. The person would not have to make any responses, except for maybe catch trials to make sure they are paying attention.

- **Stimulus Identification:** This condition requires a model, but not one that takes too much information. All it does is it identifies, on each trial, the identity of the previous stimulus.

- **Source prediction**: The most complex model would be one in which the person tries to predict the stimulus/source on the next trial. This could be in a condition in which there is or there isn;t a hazard rate (so far these measures have been used in situations where a hazard rate needs to be inferred).

The blocks below are meant to test different models that do different levels of inference:

- **Source inference model:** Model that strictly attempts to infer the source generating the stimuli
- **Source inference with fixed hazard:** Model that infers the source while considering a single, fixed hazard rate
- **Source and hazard inference - fixed hazard:** Model that infers the the source and the hazard rate, assuming that sources can change but not the hazard rate. DOes this inference with a prior over the hazard space.
- **Source and hazard inference - variable hazard:**: Model that infers the source and the hazard rate, but assumes that the hazard rate itself can also change. Also has a prior over the hazard space.

In [3]:
#Import libraries
%pylab inline
import numpy as np

Populating the interactive namespace from numpy and matplotlib


### 0) Task set up

To be clear about the task we are attempting to model, the general set up will be that a person hears tone, either in the left ear or the right ear, and has to make inferences about those tones based on the condition they are in.

This may change but depending on the instructions, participants will be told that two sources could be generating the tones:

- Source 1 generates tone 80% of the time on the left and 20 % on the right
- Source 2 generates tones 80% on the right and 20% on the left

Tone produced from each source are generated via stochastic bernoulli process.


### 1) Source inference model

First start with a simple source inference model that does not include a hazard rate. This is a simple bayesian updater that acummlates evidence for one source or the other and makes it's best guess as to which source has generated all previously heard tones on each trial.

**ADD MATH HERE**

Let $z$ be the source and $x$ be the observations on a trial $t$ - our model is simply trying to compute $p(z|x_{1:t})$ using bayes rule:

$$p(z|x_{1:t}) \propto p(x_{1:t}|z)p(z)$$

In [4]:
def source_inf(x,p):
    """
    Source inference model - infers which of two sources generated binary observationsx
    Arguments:
        - x: array of binary observations (0s and 1s)
        - p: probability p of the bernoulli processes
    Returns:
        - pz: trial by trial probability that each source is the generative source
    """
    pz = np.empty((2,len(x)))
    for i in range(len(x)):
        if i == 1:
            pz[:,i] = np.array([.5,.5])
        else:
            #These are ugly loops...I'm sure there's a way to optimize them
            for iz in range(2):
                

In [5]:
x = [1]*10
pz = np.empty((2,len(x)))
print(pz)

[[-1.72723371e-077 -1.72723371e-077  1.77863633e-322  0.00000000e+000
   0.00000000e+000  5.02034658e+175  6.95354075e-042  3.24978063e-057
   1.27796323e+165  9.51371601e-047]
 [ 1.47763641e+248  1.16096346e-028  7.69165785e+218  1.35617292e+248
   1.41530904e+161  6.00457966e-067  4.66528258e-033  6.28293892e-038
   4.30620931e-096  6.32299154e+233]]
